This create one spatiotemporal diagram for one incident

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [27]:
# could later set global variables here
minutes = 240  # 20 hours
meters = 6000 # 10 km

minute_steps = 10 
meter_steps = 100

sigma = 200 # standard was 45
tau = 45 # standard was 45

In [28]:
def phi_0(x, t, xi, ti, sigma, tau):
    return np.exp(- (np.abs(x - xi) / sigma + np.abs(t - ti) / tau))

def N(x, t, datapoints, sigma, tau):
    return sum(phi_0(x, t, xi, ti, sigma, tau) for xi, ti, _ in datapoints)

def V(x, t, datapoints, sigma, tau):
    normalization_factor = N(x, t, datapoints, sigma, tau)
    return sum(phi_0(x, t, xi, ti, sigma, tau) * vi for xi, ti, vi in datapoints) / normalization_factor

def spatiotemporal(datapoints):
    

    # Create a grid for the spatiotemporal map
    spatio_map = np.zeros((int(minutes / minute_steps), int(meters / meter_steps)))

    for i in range(0, minutes, minute_steps):
        for j in range(0, meters, meter_steps):
            spatio_map[int(i / minute_steps), int(j / meter_steps)] = V(j, i, datapoints, sigma, tau)

    return spatio_map

# Example synthetic data (xi, ti, vi)
# datapoints = [(np.random.randint(0, meters), np.random.randint(0, minutes), np.random.uniform(20, 120)) for _ in range(100)]
def create_spatio_diagram(x_t_v_data_path, incident_id):

    data1  =pd.read_csv(x_t_v_data_path)

    # print minimum of values of the data for each column
    print(data1.min())
    print(data1.max())

    datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]

    # Generate the spatiotemporal map
    spatio_map = spatiotemporal(datapoints)

    print("Contains NaN:", np.isnan(spatio_map).any())
    print("Contains Inf:", np.isinf(spatio_map).any())

    # Define the axes
    y = np.linspace(0, meters, spatio_map.shape[1])  # Location axis for the y-axis
    x = np.linspace(0, minutes, spatio_map.shape[0])  # Time axis for the x-axis

    # Create the meshgrid for plotting, now with time on the x-axis and location on the y-axis
    T, Y = np.meshgrid(x, y)

    # Visualize the spatiotemporal map with location on the y-axis and time on the x-axis
    plt.figure(figsize=(15, 8))
    cp = plt.contourf(T, Y, spatio_map.T, levels=np.linspace(spatio_map.min(), spatio_map.max(), 100), cmap='viridis')
    plt.colorbar(cp, label='V (km/h)')



    # Set the labels correctly
    plt.xlabel('Time (minutes)')
    plt.ylabel('Location (meters)')
    plt.title('Spatiotemporal Traffic Data Visualization')
    # set in the subtitle the settings for sigma and tau
    plt.suptitle(f'Sigma = {sigma}, Tau = {tau}')
    # safe figure to ./spatiodiagrams folder, with random name

    plt.savefig('/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/diagramsForModelTraining/' + incident_id + '.png')



os.system('say "full spatio finished"')




0

In [29]:
# create all the spatio diagrams for the incidents
# get incident data from ./incidentInterpolationData folder and create spatio diagrams
min_max_df = pd.DataFrame(columns=['incident_id', 'min_x', 'max_x'])

for filename in os.listdir('/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/incidentInterpolationData'):
    if filename.endswith(".csv"):
        print(filename)
        # get full path to the file
        filepath = '/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/incidentInterpolationData/' + filename
        df = pd.read_csv(filepath)
        min_x = df['x'].min()
        max_x = df['x'].max()
        print(f"minimum x for incident {filename} is {min_x}")
        print(f"maximum x for incident {filename} is {max_x}")

        # Add to min_max_df
        min_max_df = pd.concat([min_max_df, pd.DataFrame({
            'incident_id': [filename.split('.')[0]],
            'min_x': [min_x],
            'max_x': [max_x]
        })], ignore_index=True)
        
        
        incident_id = filename.split('.')[0]
        # set to filepath
        x_t_v_data_path = '/Users/vree/GithubRepos/DataScience/basFiles/3hoursFromIncidentToInterpolationData copy/incidentInterpolationData/' + filename
        create_spatio_diagram(x_t_v_data_path, incident_id)
        print("finished")
    else:
        continue

RWS02_0000167475_167475.csv
minimum x for incident RWS02_0000167475_167475.csv is 1170.1615982696133
maximum x for incident RWS02_0000167475_167475.csv is 4707.251707281296
x    1170.161598
t       0.533333
v      21.444444
dtype: float64
x    4707.251707
t     239.533333
v     111.500000
dtype: float64


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_46394/2174492482.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  min_max_df = pd.concat([min_max_df, pd.DataFrame({
/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_46394/3590372041.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Contains NaN: False
Contains Inf: False
finished
RWS02_0000166350_166350.csv
minimum x for incident RWS02_0000166350_166350.csv is 1146.1288462966631
maximum x for incident RWS02_0000166350_166350.csv is 4679.193209912745
x    1146.128846
t       0.533333
v       4.500000
dtype: float64
x    4679.193210
t     239.533333
v     114.500000
dtype: float64


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_46394/3590372041.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Contains NaN: False
Contains Inf: False
finished
RWS03_822837_1.csv
minimum x for incident RWS03_822837_1.csv is 1171.3451323176407
maximum x for incident RWS03_822837_1.csv is 4709.954945912482
x    1171.345132
t       0.900000
v       9.555556
dtype: float64
x    4709.954946
t     239.900000
v     134.000000
dtype: float64


...

/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_46394/3590372041.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Contains NaN: False
Contains Inf: False
finished
NLRWS_NLSIT002755289_1.csv
minimum x for incident NLRWS_NLSIT002755289_1.csv is 1293.2067451448058
maximum x for incident NLRWS_NLSIT002755289_1.csv is 7726.4142277947885
x    1293.206745
t       0.000000
v      37.500000
dtype: float64
x    7726.414228
t     240.000000
v     118.000000
dtype: float64


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_46394/3590372041.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Contains NaN: False
Contains Inf: False
finished
RWS02_0000166496_166496.csv
minimum x for incident RWS02_0000166496_166496.csv is 1072.319314396111
maximum x for incident RWS02_0000166496_166496.csv is 7896.215283078425
x    1072.319314
t       0.033333
v      19.250000
dtype: float64
x    7896.215283
t     239.033333
v     169.000000
dtype: float64


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_46394/3590372041.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datapoints = [(row[0], row[1], row[2]) for index, row in data1.iterrows()]


Creating maps without interpolation below

In [ ]:

# # show map without extrapolating the data, just the traces of the cameras

# def spatiotemporal_unique(datapoints):
#     # Create a grid for the spatiotemporal map
#     spatio_map = np.zeros((int(minutes / minute_steps)+2, int(meters / meter_steps)+2))
    
#     # per location the datapoints are added chronologically, therefore we can use here the minutesteps as well
#     for i in range(0, len(datapoints), minute_steps):
#         spatio_x_axis = int(datapoints[i][1]/minute_steps)
#         spatio_y_axis = int(datapoints[i][0]/meter_steps)
#         print("spatio x axis: ", spatio_x_axis, "for i = ", i)
#         print("spatio y axis: ", spatio_y_axis, "for i = ", i)
#         spatio_map[spatio_x_axis, spatio_y_axis] = V(datapoints[i][0], datapoints[i][1], datapoints, sigma, tau)
        
#     return spatio_map


# # Example synthetic data (xi, ti, vi)
# # datapoints = [(np.random.randint(0, meters), np.random.randint(0, minutes), np.random.uniform(20, 120)) for _ in range(100)]

# # Load the data
# data = pd.read_csv(x_t_v_data_path)
# datapoints = [(row[0], row[1], row[2]) for index, row in data.iterrows()]

# # Generate the spatiotemporal map
# spatio_map = spatiotemporal_unique(datapoints)

# # Define the axes
# y = np.linspace(0, meters, spatio_map.shape[1])  # Location axis for the y-axis
# x = np.linspace(0, minutes, spatio_map.shape[0])  # Time axis for the x-axis

# # Create the meshgrid for plotting, now with time on the x-axis and location on the y-axis
# T, Y = np.meshgrid(x, y)

# # Visualize the spatiotemporal map with location on the y-axis and time on the x-axis
# plt.figure(figsize=(15, 8))
# cp = plt.contourf(T, Y, spatio_map.T, levels=np.linspace(spatio_map.min(), spatio_map.max(), 100), cmap='viridis')
# plt.colorbar(cp, label='V (km/h)')

# # Set the labels correctly
# plt.xlabel('Time (minutes from 7 AM)')
# plt.ylabel('Location (km)')
# plt.title('Spatiotemporal Traffic Data Visualization')

# #save figure to ./spatiodiagrams_traces folder, with random name
# plt.savefig('./spatiodiagrams_traces/spatiotemporal' + str(random) + '.png')

# plt.show()      

# os.system('say "your program has finished"')
        
        


In [ ]:
# import os
# # For macOS
# os.system('say "your program has finished"')


